In [ ]:
!pip install transformers openpyxl requests


In [ ]:
import requests
import openpyxl
from transformers import pipeline
from google.colab import files
from datetime import datetime

# Set up the sentiment analysis pipeline
sentiment_pipeline = pipeline("text-classification", model="mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")

# Set up the endpoint and parameters for the Twitter API request
url = "https://twitter154.p.rapidapi.com/hashtag/hashtag"
querystring = {"hashtag": "#tesla", "limit": "20", "section": "top"}
headers = {
    "X-RapidAPI-Key": "**",
    "X-RapidAPI-Host": "twitter154.p.rapidapi.com"
}

# Make the API request
response = requests.get(url, headers=headers, params=querystring)

# Set the encoding to 'utf-8'
response.encoding = 'utf-8'

# Check if the request was successful
if response.status_code == 200:
    tweets_data = response.json()

    # Create a new Excel workbook
    wb = openpyxl.Workbook()
    ws = wb.active
    ws.title = 'Tesla Hashtag Tweets'

    # Write the header row
    headers = [
        'Tweet ID', 'Date', 'Tweet', 'Language', 'Retweet Count', 'Views',
        'Follower Count', 'Is Verified', 'Is Blue Verified', 'Sentiment Label', 'Sentiment Score'
    ]
    ws.append(headers)

    # Loop through each tweet in the response
    for tweet in tweets_data.get('results', []):
        try:
            # Extract data from the tweet and user object
            tweet_id = tweet.get('tweet_id', 'N/A')
            tweet_date = tweet.get('creation_date', 'N/A')
            tweet_text = tweet.get('text', 'N/A')

            # Perform sentiment analysis
            sentiment_result = sentiment_pipeline(tweet_text)
            sentiment_label = sentiment_result[0]['label']
            sentiment_score = sentiment_result[0]['score']

            # Continue extracting other data
            language = tweet.get('language', 'N/A')
            retweet_count = tweet.get('retweet_count', 'N/A')
            views = tweet.get('views', 'N/A')
            user = tweet.get('user', {})
            follower_count = user.get('follower_count', 'N/A')
            is_verified = user.get('is_verified', 'N/A')
            is_blue_verified = user.get('is_blue_verified', 'N/A')

            # Write the data to the Excel file
            ws.append([
                tweet_id,
                tweet_date,
                tweet_text,
                language,
                retweet_count,
                views,
                follower_count,
                is_verified,
                is_blue_verified,
                sentiment_label,
                sentiment_score
            ])
        except Exception as e:
            print(f"An error occurred while processing a tweet: {e}")

    # Define the Excel filename with current datetime
    xlsx_filename = f'tesla_hashtag_tweets_sentiment_{datetime.now().strftime("%Y%m%d_%H%M%S")}.xlsx'

    # Save the Excel workbook
    wb.save(xlsx_filename)

    print(f"Tweets and sentiment analysis have been saved to {xlsx_filename}")
else:
    print("Failed to fetch tweets:", response.status_code)
    files.download(xlsx_filename)

Tweets and sentiment analysis have been saved to tesla_hashtag_tweets_sentiment_20240328_094359.xlsx
